<a href="https://colab.research.google.com/github/leesk212/2021_IDEC/blob/main/IDEC_GPU_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9ry2k1fu
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9ry2k1fu
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=97bcfc860ca565c3fcce7357b19b179c9cdd358918d34f34866d53b9c4fb5684
  Stored in directory: /tmp/pip-ephem-wheel-cache-boiw46rs/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu


#include <stdio.h>
__global__ void cuda_hello(){
    printf("Hello World from GPU!\n");
}
int main() {
    cuda_hello<<<1,1>>>();
    cudaDeviceSynchronize();
    return 0;
}

Hello World from GPU!



In [ ]:
%%cu

#include <stdio.h>

__global__ void addKernel(int *c, const int *a, const int *b){
    int i = threadIdx.x;
    c[i] = a[i] + b[i];
}


int main(){
    // Variables in Host
    const int SIZE = 5;
    const int a[SIZE] = { 1,2,3,4,5 };
    const int b[SIZE] = {10,20,30,40,50};
    int c[SIZE] = {0};
    int *dev_a = 0;
    int *dev_b = 0;
    int *dev_c = 0;

    // Device (GPU) Memory allocation

    cudaMalloc((void**)&dev_c, SIZE * sizeof(int));
    cudaMalloc((void**)&dev_a, SIZE * sizeof(int)); 
    cudaMalloc((void**)&dev_b, SIZE * sizeof(int));


    // Memory copy from Host to Device
    cudaMemcpy(dev_a, a, SIZE * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, SIZE * sizeof(int), cudaMemcpyHostToDevice);

    // Computation in GPU
    addKernel<<<1, SIZE>>>(dev_c,dev_a,dev_b);

    // Result data copy from Device to Host
    cudaMemcpy(c, dev_c, SIZE * sizeof(int), cudaMemcpyHostToDevice);

    printf("{1,2,3,4,5}+{10,20,30,40,50} = {%d,%d,%d,%d,%d}\n", c[0],c[1],c[2],c[3],c[4]);
    fflush(stdout);
    cudaFree(dev_c);
    cudaFree(dev_a);
    cudaFree(dev_b);
    return 0;
}

{1,2,3,4,5}+{10,20,30,40,50} = {0,0,0,0,0}



In [ ]:
%%cu
#include <stdio.h>

void addKernel(int *dst,int aVal, int bVal){
    *dst = aVal + bVal;
}

int main(){
    // Variables in Host
    const int SIZE = 5;
    const int a[SIZE] = { 1,2,3,4,5 };
    const int b[SIZE] = {10,20,30,40,50};
    int c[SIZE] = {0};
    

    for(int i = 0; i<SIZE; i++){
        //c[i] = a[i] + b[i];
        addKernel(&c[i],a[i],b[i]);
    }
    printf("{1,2,3,4,5}+{10,20,30,40,50} = {%d,%d,%d,%d,%d}\n", c[0],c[1],c[2],c[3],c[4]);
    return 0;
}

{1,2,3,4,5}+{10,20,30,40,50} = {11,22,33,44,55}

